# Numpy

Материалы:
* Макрушин С.В. "Лекция 1: Библиотека Numpy"
* https://numpy.org/doc/stable/user/index.html
* https://numpy.org/doc/stable/reference/index.html

## Лабораторная работа №1

Замечание: при решении данных задач не подразумевается использования циклов или генераторов Python, если в задании не сказано обратного. Решение должно опираться на использования функционала библиотеки `numpy`.

1. Файл `minutes_n_ingredients.csv` содержит информацию об идентификаторе рецепта, времени его выполнения в минутах и количестве необходимых ингредиентов. Считайте данные из этого файла в виде массива `numpy` типа `int32`, используя `np.loadtxt`. Выведите на экран первые 5 строк массива.

In [1]:
import numpy as np

dataset = np.loadtxt('data/minutes_n_ingredients.csv', skiprows=1, delimiter=',', dtype=np.int32)
print(dataset[:5])

[[127244     60     16]
 [ 23891     25      7]
 [ 94746     10      6]
 [ 67660      5      6]
 [157911     60     14]]


2. Вычислите среднее значение, минимум, максимум и медиану по каждому из столбцов, кроме первого.

In [2]:
cols = [1, 2]
average = np.mean(dataset[:, cols], axis = 0, dtype = np.int32)
mininal = np.amin(dataset[:, cols], axis=0)
max = np.amax(dataset[:, cols], axis=0)
median = np.median(dataset[:, cols], axis=0)
print(average)
print(mininal)
print(max)
print(median)

[-21348      9]
[0 1]
[2147483647         39]
[40.  9.]


3. Ограничьте сверху значения продолжительности выполнения рецепта значением квантиля $q_{0.75}$. 

In [3]:
q = np.quantile(dataset[:, 1], 0.75)
dataset[:, 1] = dataset[:, 1].clip(max=q)

4. Посчитайте, для скольких рецептов указана продолжительность, равная нулю. Замените для таких строк значение в данном столбце на 1.

In [4]:
amount = dataset[:, 1].size - np.count_nonzero(dataset[:, 1])
print(amount)
dataset[:, 1][dataset[:, 1] == 0 ] = 1

479


5. Посчитайте, сколько уникальных рецептов находится в датасете.

In [5]:
cols = [1, 2]
n = len(np.unique(dataset[:, cols]))
print(n)

65


6. Сколько и каких различных значений кол-ва ингредиентов присутвует в рецептах из датасета?

In [6]:
n = len(np.unique(dataset[:, 2]))
unique = np.unique(dataset[:, 2])
print(n)
print(unique)

37
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 37 39]


7. Создайте версию массива, содержащую информацию только о рецептах, состоящих не более чем из 5 ингредиентов.

In [7]:
index = np.where(dataset[:, 2] > 5 )
np.delete(dataset, index, 0)

array([[446597,     15,      5],
       [204134,      5,      3],
       [ 25623,      6,      4],
       ...,
       [ 52088,     60,      5],
       [128811,     15,      4],
       [370915,      5,      4]], dtype=int32)

8. Для каждого рецепта посчитайте, сколько в среднем ингредиентов приходится на одну минуту рецепта. Найдите максимальное значение этой величины для всего датасета

In [8]:
print(np.amax(dataset[:, 2] / dataset[:, 1]))

24.0


9. Вычислите среднее количество ингредиентов для топ-100 рецептов с наибольшей продолжительностью

In [9]:
sort = dataset[dataset[:, 1]. argsort ()[::-1]]
sorted = sort[:100][:, 2]
print(np.mean(sorted))

9.96


10. Выберите случайным образом и выведите информацию о 10 различных рецептах

In [10]:
rows, columns = dataset.shape
cols = [np.random.randint(0, rows+1) for i in range(10)]
print(dataset[cols, :])

[[152800     40     14]
 [312629     65      5]
 [ 81391     45     14]
 [384591     60     15]
 [233469     65     13]
 [134165     65      9]
 [ 28896     15      5]
 [207736     20      5]
 [188726     35      9]
 [259942     65      8]]


11. Выведите процент рецептов, кол-во ингредиентов в которых меньше среднего.

In [11]:
av = np.mean(dataset[:, 2])
set = dataset[:, 2] < av
res = dataset[:, 2][set].size * 100 / dataset[:, 2].size
print(res)

58.802


12. Назовем "простым" такой рецепт, длительность выполнения которого не больше 20 минут и кол-во ингредиентов в котором не больше 5. Создайте версию датасета с дополнительным столбцом, значениями которого являются 1, если рецепт простой, и 0 в противном случае.

In [12]:
rows, columns = dataset.shape
zero = np.zeros((rows, 1), dtype=np.int32)
dataset = np.concatenate((dataset, zero), axis=1)
i = np.where((dataset[:, 2] <= 5) & (dataset[:, 1] <= 20))
dataset[i, 3] = 1
print(dataset)


[[127244     60     16      0]
 [ 23891     25      7      0]
 [ 94746     10      6      0]
 ...
 [498432     65     15      0]
 [370915      5      4      1]
 [ 81993     65     14      0]]


13. Выведите процент "простых" рецептов в датасете

In [13]:
x = dataset[:, 3]
set = x == 1
res = x[set].size * 100 / x.size
print(res)

9.552


14. Разделим рецепты на группы по следующему правилу. Назовем рецепты короткими, если их продолжительность составляет менее 10 минут; стандартными, если их продолжительность составляет более 10, но менее 20 минут; и длинными, если их продолжительность составляет не менее 20 минут. Создайте трехмерный массив, где нулевая ось отвечает за номер группы (короткий, стандартный или длинный рецепт), первая ось - за сам рецепт и вторая ось - за характеристики рецепта. Выберите максимальное количество рецептов из каждой группы таким образом, чтобы было возможно сформировать трехмерный массив. Выведите форму полученного массива.